# Instalação do pydantic

In [ ]:
pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.6 MB/s eta 0:00:00


# Importação das Bibliotecas necessárias

In [ ]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

Instruções para validação das expressões regulares com re para a senha e o nome, definidos em suas respectivas variáveis

In [ ]:
# Validação de senha para que ela contenha ao menos 8 caractéres, uma minúscula,
# uma maiúscula e um número
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")

# Validação do nome para que ele tenha apenas letras e ao menos 2 caracteres
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

# Definição das **classes** Role e User, usando pydantic para criar modelos de dados com validação

In [ ]:
# Define uma númeração para cada papel, permitindo cobiná-los com bitwise
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# A classe user define o modelo de dados para o usuário, com nome, email, senha e papel
# O Field permite adição de dados como exemplos e descrições
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

# Este método Field_validator usa a variável para nome definida anteriormente,
# Verificando se contém apenas letras e até dois dígitos
    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

# Este fieldvalidator se aplica ao papel, podendo receber um numero int, string
# ou enum number, e converte qualquer um destes para um enum number relacionado ao
# Role, se o input for inválido ele retorna um erro
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

# Este método model_validator checa se foram preencidos o nome e a senha, garante
# que a senha não seja igual ao nome, valida a senha, utiliza um hash para atestar
# sua autenticidade e se ocorrer qualquer erro ele retorna um ValueError
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# Definição da função que valida os dados para criar um objeto User baseado nas regras da classe User

In [ ]:
# a função validate recebe os dados que devem corresponder aos fields da classe User
# o try verifica a possível existência de erros
# o método model_validate é a ação principal, criando um objeto user após validar
# seus dados, se ocorrer um erro retorna uma mensagem de erro e se der certo mostra
# o user
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

# Função para testar os dados e validar o usuário se estiver tudo correto

In [ ]:
# Usa o dicionário test data com dados ficticios para teste rotulados em:
# good data, bad data e bad role(bons dados, dados ruins e papeis ruins)
def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )
# o loop for itera por cada item no dicionário test data, separando pelos rótulos
# dos dados e o validate faz a validação do seu conteúdo
    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()


if __name__ == "__main__":
    main()